In [1]:
from dataloader.BSD500 import BSD500
import torch
from R_network_relax import RNet
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 5  


val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_5_relaxed_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]


exp = "/home/pourya/mm_final/exps/sigma5/64_2_7_1e-3_relaxed"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:1'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()


model.eigenimage = model.eigenimage.to(device)
lip, model.eigenimage = model.W1.cal_lip(model.eigenimage, 1000)
model.alpha = 1 / lip
model.lmbda = torch.nn.Parameter(torch.ones(1, 1).to(device) * p1)


model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)

In [5]:
def safi_denoise(y):
    s = 1e-3
    n = 5
    e = 1e-5
    q = (e/s) ** (1/n)
    tols = [s * q**(i) for i in range(n+1)]
    n_out = 10
    for _ in range(n_out-(n+1)):
        tols.append(e)
    with torch.no_grad(): 
        n_in = 100  
        c_k = model.prox_denoise_no_mask(y, y, n_in, tols[0], check_tol=True)
        for it in range(n_out - 1):   
            model.cal_mask(c_k)
            c_k_new = model.prox_denoise_with_mask(y, c_k, n_in, tols[it+1], check_tol=True)
            rel_err_out = (torch.norm(c_k - c_k_new) / torch.norm(c_k)).item() 
            sc = "{:0.1e}".format(rel_err_out)
            c_k = c_k_new
    
    return c_k

In [6]:
i = 0
psnrs = list()

with torch.no_grad():
    for img in test_dataset:
        i = i + 1
        if True:
            gt = img.to(device)[None, :, :, :]
            noisy_image = (img.to(device) + (5/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
            denoised =safi_denoise(noisy_image)
            psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
            psnrs.append(psnr)
            print(i, psnr)

1 38.35
2 37.1
3 39.52
4 38.29
5 38.56
6 36.45
7 38.19
8 37.85
9 39.26
10 37.95
11 37.27
12 36.04
13 38.22
14 36.3
15 37.53
16 39.05
17 37.04
18 37.24
19 38.6
20 38.08
21 40.99
22 35.36
23 35.56
24 35.51
25 37.8
26 38.08
27 37.33
28 40.72
29 36.72
30 37.9
31 37.12
32 39.06
33 38.22
34 34.82
35 37.6
36 37.58
37 37.65
38 37.02
39 40.6
40 35.84
41 37.84
42 41.1
43 38.47
44 39.27
45 36.33
46 38.3
47 37.7
48 36.2
49 35.27
50 45.34
51 37.13
52 39.76
53 41.67
54 39.01
55 37.79
56 36.64
57 36.43
58 36.62
59 34.95
60 36.17
61 37.2
62 36.94
63 38.34
64 35.07
65 38.4
66 41.48
67 40.29
68 39.06


In [7]:
print(np.round(np.mean(np.array(psnrs)), 2), np.round(np.std(np.array(psnrs)), 2))

37.9 1.82
